In [1]:
import numpy as np
import pandas as pd
from osgeo import gdal
import os
import time
import xlwt

In [2]:
class Dataset:
    def __init__(self, in_file):
        self.in_file = in_file  # Tiff或者ENVI文件

        dataset = gdal.Open(self.in_file)
        self.XSize = dataset.RasterXSize  # 网格的X轴像素数量
        self.YSize = dataset.RasterYSize  # 网格的Y轴像素数量
        self.Bands = dataset.RasterCount  # 波段数
        self.GeoTransform = dataset.GetGeoTransform()  # 投影转换信息
        self.ProjectionInfo = dataset.GetProjection()  # 投影信息

#     def get_data(self, band):
#         #band: 读取第几个通道的数据
#         dataset = gdal.Open(self.in_file)
#         band = dataset.GetRasterBand(band)
#         data = band.ReadAsArray()
#         return data
    
    def get_data(self):
        #band: 读取第几个通道的数据
        dataset = gdal.Open(self.in_file)
        data = dataset.ReadAsArray(0,0,self.XSize,self.YSize)
        return data
    

    def get_lon_lat(self):
        #获取经纬度信息
        gtf = self.GeoTransform
        x_range = range(0, self.XSize)
        y_range = range(0, self.YSize)
        x, y = np.meshgrid(x_range, y_range)
        lon = gtf[0] + x * gtf[1] + y * gtf[2]
        lat = gtf[3] + x * gtf[4] + y * gtf[5]
        
        lon_lat=[]
        for (longitude,latitude) in zip(lon,lat):
            lon_lat.append(list(zip(longitude,latitude)))
            
        return np.array(lon_lat)
    
    def new_dataset(self,data,lon_lat):
        new_dataset=[]
        for i in range(self.YSize):
            for j in range(self.XSize):
                x1 = lon_lat[i,j]
                x2 = data[:,i,j]
#                 x2 = data[i,j]
                x=np.hstack((x1,x2))
                new_dataset.append(x)
#         for (i,j) in zip(lon_lat[i,j],data[:,i,j]):
#             new_dataset.append(list(zip(i,j)))
            
        return np.array(new_dataset)

In [3]:
# 以下代码演示读取E:/data/dataset.tif的第一个通道的数据，并且获取经纬度信息
start_time = time.time()
dir_path = r"D:\Desktop"
filename = "scale-offset-linear.tif"
file_path = os.path.join(dir_path, filename)
dataset = Dataset(file_path)

data = dataset.get_data( )  # 获取第一个通道的数据

lon_lat = dataset.get_lon_lat()  # 获取经纬度信息longitude, latitude

new_dataset=dataset.new_dataset(data,lon_lat)

# # EVI_108
writer = pd.ExcelWriter(r"D:\Desktop\scale-offset-linear.xls")
# pd.DataFrame(new_dataset).to_excel(writer,sheet_name='LAI',na_rep=0,\
#                                    index=False,header=['lon','lat']+['LAI']*92)
pd.DataFrame(new_dataset).to_excel(writer,na_rep=0,index=False,\
                                   header=['lon','lat','p0','p1','p2','p3','p4','p5','item'])
writer.save()
writer.close()

D:\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3361: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
import sys
print(sys.getsizeof(new_dataset))

839160


In [4]:
dir_path = r"D:\Desktop"
filename = "scale-offset-LM.tif"
file_path = os.path.join(dir_path, filename)
dataset = Dataset(file_path)

data = dataset.get_data( )  # 获取第一个通道的数据

lon_lat = dataset.get_lon_lat()  # 获取经纬度信息longitude, latitude

new_dataset=dataset.new_dataset(data,lon_lat)
new_dataset.shape

writer = pd.ExcelWriter(r"D:\Desktop\scale-offset-LM.xls")


pd.DataFrame(new_dataset).to_excel(writer,na_rep=0,index=False,\
                                   header=['lon','lat','scale','offset'])
# ,sheet_name='biubiu'
writer.save()
writer.close()